In [102]:
import twitter
import json
#---------------------------------------------
# Define a Function to Login Twitter API
def oauth_login():
    # Prof. Paffenroth has a developer account for the class.  He will provide the Twitter access tokens for
    # each team
    # See https://developer.twitter.com/docs/auth/oauth for more information 
    # on Twitter's OAuth implementation.
    
    CONSUMER_KEY = 'QoTiAk8g8Hm9Zhejyt1GxF9Bp'
    CONSUMER_SECRET ='Lj6Hg1XlDcKeT2pFNS330vN7RqgRzxHpsUovESMd9braCZ4WGO'
    OAUTH_TOKEN = '571213367-dPdnChuVRjwRHoqfH0pi15IbgGhJmRoKxJjrP9wD'
    OAUTH_TOKEN_SECRET = '3cp8koRc8EKcVn3DkI0HoGquXJWMnAYgU7pzMiMnNq3jq'
    
    auth = twitter.oauth.OAuth(OAUTH_TOKEN, OAUTH_TOKEN_SECRET,
                               CONSUMER_KEY, CONSUMER_SECRET)
    
    twitter_api = twitter.Twitter(auth=auth)
    return twitter_api

twitter_api = oauth_login()

#----------------------------------------------
# Your code starts here
#   Please add comments or text cells in between to explain the general idea of each block of the code.
#   Please feel free to add more cells below this cell if necessary

# XXX: Set this variable to a trending topic,
# or anything else for that matter. The example query below
# was a trending topic when this content was being developed
# and is used throughout the remainder of this chapter.
q = 'netflix'
count = 100
# See https://dev.twitter.com/docs/api/1.1/get/search/tweets
search_results = twitter_api.search.tweets(q=q, count=count)
statuses = search_results['statuses']
# Iterate through 5 more batches of results by following the cursor
for _ in range(10):
    try:
        next_results = search_results['search_metadata']['next_results']
    except(KeyError): # No more results when next_results doesn't exist
        break

     # Create a dictionary from next_results, which has the following form:
     # ?max_id=313519052523986943&q=NCAA&include_entities=1
    kwargs = dict([ kv.split('=') for kv in next_results[1:].split("&") ])

    search_results = twitter_api.search.tweets(**kwargs)

    statuses += search_results['statuses']

status_texts = [ status['text'] for status in statuses ]

with open('data.json', 'w', encoding='utf-8') as f:
    json.dump(status_texts, f, ensure_ascii=False, indent=4)
    
# Explore the first 5 items for each...
# print (json.dumps(status_texts[0:5], indent=1))
print(len(statuses))


1100


In [103]:
# Word Count
from prettytable import PrettyTable
from collections import Counter

with open('data.json', 'r', encoding = 'utf8') as twitter_statuses:
    twitter_text = twitter_statuses.read()
    words = twitter_text.split()
    
    
    
    c = Counter(words)
    common_words = c.most_common()[:100]
    
    pt = PrettyTable(field_names=['Word', 'Count'])
    [ pt.add_row(kv) for kv in common_words ]
    pt.align['Word'], pt.align['Count'] = 'l', 'r'
    
    print(pt)

+---------------+-------+
| Word          | Count |
+---------------+-------+
| "RT           |   632 |
| Netflix       |   361 |
| a             |   340 |
| de            |   258 |
| the           |   187 |
| to            |   147 |
| you           |   146 |
| on            |   142 |
| que           |   137 |
| netflix       |   129 |
| is            |   107 |
| I             |   104 |
| and           |   100 |
| e             |    91 |
| me            |    90 |
| ass           |    80 |
| y             |    78 |
| of            |    71 |
| this          |    67 |
| in            |    62 |
| for           |    62 |
| still         |    61 |
| with          |    60 |
| para          |    59 |
| no            |    57 |
| o             |    56 |
| @netflix      |    55 |
| la            |    54 |
| na            |    53 |
| watch         |    53 |
| netflix:      |    52 |
| The           |    52 |
| if            |    51 |
| da            |    48 |
| @nickxnl:     |    47 |
| “are      

In [94]:
retweets = [
 # Store out a tuple of these three values ...
 (status['retweet_count'],
 status['retweeted_status']['user']['screen_name'],
 status['text'])

 # ... for each status ...
 for status in statuses if 'retweeted_status' in status
 ]


pt = PrettyTable(field_names=['Count', 'Screen Name', 'Text'])

[ pt.add_row(row) for row in sorted(set(retweets), reverse=True)[:10] ]

pt.max_width['Text'] = 50

pt.align= 'l'

print(pt)


+-------+-----------------+----------------------------------------------------+
| Count | Screen Name     | Text                                               |
+-------+-----------------+----------------------------------------------------+
| 82945 | MarieAnnUK      | RT @MarieAnnUK: That awkward moment when you're    |
|       |                 | being such a bastard Putin has to intervene.       |
|       |                 | https://t.co/kpk40NBvPj                            |
| 55016 | Iowlifee        | RT @Iowlifee: How tf do drug dealers get more jail |
|       |                 | time than rapists?? I’d rather have Pablo the Plug |
|       |                 | as a neighbor than Randy the Rapist                |
| 22640 | chaselyons      | RT @chaselyons: I FaceTimed her and heard PS4      |
|       |                 | music in the background https://t.co/o9AlAhtTrk    |
| 7623  | tomgara         | RT @tomgara: A man sued his employer for racial    |
|       |                 | 

In [105]:
import sys
import time
from urllib.request import URLError
from http.client import BadStatusLine
import json
import twitter

def make_twitter_request(twitter_api_func, max_errors=10, *args, **kw): 
    
    # A nested helper function that handles common HTTPErrors. Return an updated
    # value for wait_period if the problem is a 500 level error. Block until the
    # rate limit is reset if it's a rate limiting issue (429 error). Returns None
    # for 401 and 404 errors, which requires special handling by the caller.
    def handle_twitter_http_error(e, wait_period=2, sleep_when_rate_limited=True):
    
        if wait_period > 3600: # Seconds
            print >> sys.stderr, 'Too many retries. Quitting.'
            raise e
    
        # See https://dev.twitter.com/docs/error-codes-responses for common codes
    
        if e.e.code == 401:
            print >> sys.stderr, 'Encountered 401 Error (Not Authorized)'
            return None
        elif e.e.code == 404:
            print >> sys.stderr, 'Encountered 404 Error (Not Found)'
            return None
        elif e.e.code == 429: 
            print >> sys.stderr, 'Encountered 429 Error (Rate Limit Exceeded)'
            if sleep_when_rate_limited:
                print >> sys.stderr, "Retrying in 15 minutes...ZzZ..."
                sys.stderr.flush()
                time.sleep(60*15 + 5)
                print >> sys.stderr, '...ZzZ...Awake now and trying again.'
                return 2
            else:
                raise e # Caller must handle the rate limiting issue
        elif e.e.code in (500, 502, 503, 504):
            print >> sys.stderr, 'Encountered %i Error. Retrying in %i seconds' % \
                (e.e.code, wait_period)
            time.sleep(wait_period)
            wait_period *= 1.5
            return wait_period
        else:
            raise e

    # End of nested helper function
    
    wait_period = 2 
    error_count = 0 

    while True:
        try:
            return twitter_api_func(*args, **kw)
        except(twitter.api.TwitterHTTPError, e):
            error_count = 0 
            wait_period = handle_twitter_http_error(e, wait_period)
            if wait_period is None:
                return
        except(URLError, e):
            error_count += 1
            print >> sys.stderr, "URLError encountered. Continuing."
            if error_count > max_errors:
                print >> sys.stderr, "Too many consecutive errors...bailing out."
                raise
        except(BadStatusLine, e):
            error_count += 1
            print >> sys.stderr, "BadStatusLine encountered. Continuing."
            if error_count > max_errors:
                print >> sys.stderr, "Too many consecutive errors...bailing out."
                raise

# Sample usage

twitter_api = oauth_login()

# See https://dev.twitter.com/docs/api/1.1/get/users/lookup for 
# twitter_api.users.lookup

response = make_twitter_request(twitter_api.users.lookup, 
                                screen_name="WPI")

#print(json.dumps(response, indent=1))

In [109]:
#----------------------------------------------
# Your code starts here
#   Please add comments or text cells in between to explain the general idea of each block of the code.
#   Please feel free to add more cells below this cell if necessary

from functools import partial
from sys import maxsize

def get_friends_followers_ids(twitter_api, screen_name=None, user_id=None,
                              friends_limit=maxsize, followers_limit=maxsize):
    
    # Must have either screen_name or user_id (logical xor)
    assert (screen_name != None) != (user_id != None), \
    "Must have screen_name or user_id, but not both"
    
    get_friends_ids = partial(make_twitter_request, twitter_api.friends.ids, 
                              count=5000)
    get_followers_ids = partial(make_twitter_request, twitter_api.followers.ids, 
                                count=5000)#how many to search at a time

    friends_ids, followers_ids = [], []
    
    for twitter_api_func, limit, ids, label in [
                    [get_friends_ids, friends_limit, friends_ids, "friends"], 
                    [get_followers_ids, followers_limit, followers_ids, "followers"]
                ]:
        
        if limit == 0: continue
        
        cursor = -1
        while cursor != 0:
        
            # Use make_twitter_request via the partially bound callable...
            if screen_name: 
                response = twitter_api_func(screen_name=screen_name, cursor=cursor)
            else: # user_id
                response = twitter_api_func(user_id=user_id, cursor=cursor)

            if response is not None:
                ids += response['ids']
                cursor = response['next_cursor']
        
#             print >> sys.stderr, 'Fetched {0} total {1} ids for {2}'.format(len(ids), 
#                                                     label, (user_id or screen_name))
            print(f"Fetched {len(ids)} total {label} ids for {user_id or screen_name}")
        
            # XXX: You may want to store data during each iteration to provide an 
            # an additional layer of protection from exceptional circumstances
        
            if len(ids) >= limit or response is None:
                break

    # Do something useful with the IDs, like store them to disk...
    return friends_ids[:friends_limit], followers_ids[:followers_limit]
    # Sample usage

twitter_api = oauth_login()

friends_ids, followers_ids = get_friends_followers_ids(twitter_api, 
                                                       screen_name="WPI", 
                                                       friends_limit=20, 
                                                       followers_limit=20)
print(friends_ids)
print(followers_ids)

Fetched 5000 total friends ids for WPI
Fetched 5000 total followers ids for WPI
[19237643, 61004371, 16034244, 209218225, 847822776941596672, 14935223, 1616534552, 337470700, 1091384710708805634, 1171200166252220416, 837664231101239297, 57494095, 28166890, 632652795, 17089636, 193442384, 291838971, 14445094, 3386602930, 2929982877]
[88556930, 1178347550, 963071686013833216, 1060297676124835841, 833831655194370048, 2444314111, 1220167385577132037, 908056640410324992, 401936631, 629390104, 481096007, 820226104476762112, 914070222, 1218665338247766017, 931211556528054273, 1049459934151106560, 711365184, 1219682102976962565, 1062876713418293248, 2579258504]
